In [2]:
from OMPython import ModelicaSystem, OMCSessionZMQ
import numpy as np
import random
import os
import string
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
def load_model(folder_path, file_name, class_name):
    file_path = folder_path + "/" + file_name
    dependencies = ["Modelica"]
    for file in os.listdir(folder_path):
        if file.endswith(".mo") and not file == file_name:
            dependencies.append(folder_path + "/" + file)
    return ModelicaSystem(file_path, class_name, dependencies)

In [3]:
 def get_undefined_parameters(model):
        res = []
        for param, value in model.getParameters().items():
            if value is None and (param.startswith("init_c") or param.startswith("rates_c")):
                res.append(param)
        return res

In [4]:
def initiate_rates(model, lo, hi):
    undef = get_undefined_parameters(model)
    for param in undef:
        if param.startswith("rates_c"):
            mean = (hi - lo)/2
            std = mean
            value = abs(np.random.normal(mean, std))
            model.setParameters(**{param: value})

In [5]:
def initiate_values(model, lo, hi):
    undef = get_undefined_parameters(model)
    for param in undef:
        if param.startswith("init_c"):
            value = np.random.uniform(lo, hi)
            model.setParameters(**{param: value})

In [6]:
# Ritorna un oggetto Pandas DataFrame con all'interno i datapoints ({istante, valore}) delle variabili simulate del modello

def get_solutions(model, variables=[]):
    # recupera la "linea temporale"
    time = model.getSolutions("time")
    # se variables è vuoto, mettici i nomi di tutte le variabili del sistema
    if len(variables) == 0:
        variables = model.getSolutions()
    
    # costruisce il dataframe con solo la prima variabile
    solutions = [model.getSolutions(var) for var in variables]
    df = pd.DataFrame({"time": time, "value": solutions[0], "variable": variables[0]})
    
    # appende al dataframe tutti i dati delle altre variabili
    for i in range(1, len(variables)):
        df = df.append(
            pd.DataFrame({"time": time, "value": solutions[i], "variable": variables[i]}),
            ignore_index=True
        )
    return df

In [7]:
# Prende in input un DataFrame pandas in cui vi sia la colonna "time", e una lista di variabili
# Plotta il moto delle variabili su un grafico in cui le ascisse sono l'asse del tempo e le ordinate l'asse dei valori
# ATTENZIONE: potrebbe perdere di significato se si cerca di plottare variabili in intervalli di valori di dimensione diversa

def plot_solutions(solution_frame, variables=[]):
    if len(variables) == 0:
        sns.lineplot(data=solution_frame, x="time", y="value", hue="variable")
    else:
        sns.lineplot(data=solution_frame[solution_frame['variable'].isin(variables)], x="time", y="value", hue="variable")

my_model = load_model(".", "biosystem.mo", "Biosystem")

initiate_rates(my_model, 10e-2, 10e5)
initiate_values(my_model, 10e-10, 10e-5)

my_model.setSimulationOptions(stopTime=20)
my_model.setSimulationOptions(stepSize=10e-3)

my_model.simulate()

atm = get_solutions(my_model, ["c_1.ATM"])

print(atm)